In [3]:
from dotenv import load_dotenv
from portia.cli import CLIExecutionHooks
from portia import *
import requests
import xml.etree.ElementTree as ET
from pydantic import BaseModel, Field
from typing import Generic, TypeVar, List, ClassVar
from notion_client import Client
import os
load_dotenv(override=True)

# Fetch the Notion API key
notion_api_key = os.getenv("NOTION_API_KEY")
notion_parent_id = os.getenv("NOTION_PARENT_ID")

# Initialize the Notion client
notion = Client(auth=notion_api_key)


In [2]:
topic = ["Classifier Free Guidance"]

class YoutubeToolSchema(BaseModel): 

    """Input Schema for YoutubeTool."""
    video_preference: bool = Field(description='Whether or not the user wants video links.')
    topic: str = Field(description="The topic to search for on YouTube.")
    max_results: int = Field(default=3, description="The maximum number of results to return.")

class YoutubeTool(Tool[List[str]]):

    """ Ask the user if they would like video suggestions to supplement their learning"""

    id: ClassVar[str] = "youtube_tool"
    name: ClassVar[str] = "YouTube Tool"
    description: ClassVar[str] = ""
    args_schema = YoutubeToolSchema
    output_schema: ClassVar[tuple[str, str]] = (
        "List[str]",
        "A list of urls for video learning resources on YouTube"
    )

    def run(self, context: ToolRunContext, topic: list[str]) -> str:

        SEARCH_QUERY = "Explain {topic}"
        url = f'https://www.googleapis.com/youtube/v3/search?part=snippet&type=video&q={SEARCH_QUERY}&maxResults={MAX_RESULTS}&key={API_KEY}'
        response = requests.get(url).json()


In [4]:
from typing import Generic, TypeVar, List, ClassVar, Dict
from portia import * 

class NotionToolSchema(BaseModel):
    """Input schema for ArXiv Tool"""
    topics: list[str] = Field(..., description="The topic to learn about")

# Step 2: Define the Tool
class NotionTool(Tool[None]):
    """Creates and populates a Notion Board and subpages for learning"""

    ### Eventually add in podcasts, youtube videos, further reading
    ### As well as 

    id: ClassVar[str] = "notion_tool"
    name: ClassVar[str] = "Notion Tool"
    description: ClassVar[str] = "Create and populate Notion Learning Plan."
    args_schema = NotionToolSchema
    output_schema: ClassVar[tuple[str, str]] = (
        "list[dict[str, str]]",
        "A list of topics to create Notion pages and learning plans for."
    )

    def run(self, context: ToolRunContext, topics: list[str]) -> str:
        
        """Run the Notion Tool."""
        notion.pages.update(page_id=notion_parent_id,
        properties = {
            "title": [
                {
                    "type": "text",
                    "text": {"content": "Learning Boards"}
                }
            ]
        })

        # Step 1: Create subpages for each topic
        for topic in topics:
            notion.pages.create(
                parent={"type": "page_id", "page_id": notion_parent_id},
                properties={
                    "title": [
                        {
                            "type": "text",
                            "text": {"content": topic}
                        }
                    ]
                },
                children=[
                    {
                        "object": "block",
                        "type": "paragraph",
                        "paragraph": {
                            "rich_text": [
                                {
                                    "type": "text",
                                    "text": {
                                        "content": f"This page is about {topic}."
                                    }
                                }
                            ]
                        }
                    }
                ]
            )
        
        # Step 2: Build a clean checkbox list with no links
        checkbox_blocks = []

        for topic in topics:
            checkbox_blocks.append({"object": "block", "type": "to_do",
                "to_do": {"rich_text": [
                        {
                            "type": "text",
                            "text": {"content": topic}
                        }
                    ],
                    "checked": False}})

        # Step 3: Add a section title + checklist to the main page
        notion.blocks.children.append(
            block_id=notion_parent_id,
            children=[
                {
                    "object": "block",
                    "type": "heading_2",
                    "heading_2": {
                        "rich_text": [
                            {
                                "type": "text",
                                "text": {"content": "📈 Progress Tracker"}
                            }
                        ]
                    }
                },
                *checkbox_blocks
            ]
        )

        return "Notion has been updated!"

In [5]:
tool = NotionTool()
results = tool.run(None, topics)